In [ ]:
# following tutorial : https://starknetpy.readthedocs.io/en/latest/guide.html

In [20]:
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.account.account import Account
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.net.signer.stark_curve_signer import KeyPair
import json
import os
from starknet_py.contract import Contract
from starknet_py.cairo.felt import encode_shortstring
from starknet_py.common import create_sierra_compiled_contract

In [21]:
NODE_URL = "https://free-rpc.nethermind.io/sepolia-juno"
ACCOUNTS_PATH = os.path.join("..", "sepolia.json")

CONTRACT_DIR = os.path.join("..", "contracts")
CONTRACT_COMPILED_PATH = os.path.join(CONTRACT_DIR, "oracle_consensus_OracleConsensusND.compiled_contract_class.json")
CONTRACT_CLASS_PATH = os.path.join(CONTRACT_DIR, "oracle_consensus_OracleConsensusND.contract_class.json")

In [22]:
DECLARED_CONTRACT = "0x01336fa7c870a7403aced14dda865b75f29113230ed84e3a661f7af70fe83e7b"
DEPLOYED_CONTRACT = "0x03c750c88c20098a2c29ae47b10b3faca61603ddb8ba23e693c3675b0c3647ab"

In [23]:
client = FullNodeClient(node_url=NODE_URL)

## Connect accounts
---

In [24]:
def retrieve_accounts():
    with open(ACCOUNTS_PATH, 'r') as file:
        data = json.load(file)
    
    addresses = data['addresses']
    keys = data['private_keys']

    return addresses, keys

accounts_addresses, accounts_keys = retrieve_accounts()

In [25]:
accounts = [
    Account(
        client=client,
        address=address,
        key_pair=KeyPair.from_private_key(key),
        chain=StarknetChainId.SEPOLIA
    ) for address, key in zip(accounts_addresses, accounts_keys)
]

len(accounts)

7

## Declare / Deploy Contract

---

In [28]:
def get_built_contract() -> dict :
    result = dict()

    with open(CONTRACT_COMPILED_PATH, 'r', encoding='utf-8') as file:
        result['compiled'] = file.read()

    with open(CONTRACT_CLASS_PATH, 'r', encoding='utf-8') as file:
        result['class_hash'] = file.read()

    result['abi'] = create_sierra_compiled_contract(
        compiled_contract=result['class_hash']
    ).parsed_abi

    return result

abi = get_built_contract()['abi']

In [ ]:
async def declare():
    account = accounts[0]

    contract_file_content = get_built_contract()
    
    declare_transaction = await account.sign_declare_v3(
        contract_file_content['compiled'],
        max_fee=int(1e16)
    )

    resp = await account.client.declare(transaction=declare_transaction)
    await account.client.wait_for_tx(resp.transaction_hash)

    print(f"declared at : {resp.class_hash}")

    return resp.class_hash

async def deploy(abi, class_hash):
    contract_file_content = get_built_contract()
    
    constructor_args = {
        "admins" : accounts_addresses[:3],

        "enable_oracle_replacement" : True,
        "required_majority" : 2,
        "n_failing_oracles" : 2, 
        
        "constrained" : True,
        "dimension" : 2,
        "oracles": accounts_addresses,
    }

    deploy_result = await Contract.deploy_contract_v1(
        account=accounts[0],
        class_hash=class_hash,
        abi=abi,
        constructor_args=constructor_args,
        max_fee=int(1e16),
    )

    await deploy_result.wait_for_acceptance()

    return deploy_result.deployed_contract()

## Use Existing contract

---

In [31]:
# contract = Contract(address=DEPLOYED_CONTRACT, abi, provider=accounts[0])
# contract = await Contract.from_address(provider=accounts[1], address=DEPLOYED_CONTRACT)